# ReMi-DAS: Refraction Microtremor Processing for DAS

This Jupyter notebook demonstrates the application of the **ReMi-DAS** workflow for shear-wave velocity profiling using DAS data. The notebook guides users through a series of processing steps adapted from conventional Refraction Microtremor (ReMi) techniques, including:

- Reading and preprocessing DAS data
- Slowness-frequency (p-f) transformation
- Extraction of Rayleigh wave dispersion curves

The ReMi-DAS implementation uses the functionality of the [**DASCore**](https://github.com/DASDAE/dascore) package for signal processing.

### References

- McMechan, G.A. and Yedlin, M.J., 1981. Analysis of dispersive waves by wave field transformation. *Geophysics*, 46(6), pp.869-874.  
- Louie, J.N., 2001. Faster, better: shear-wave velocity to 100 meters depth from refraction microtremor arrays. *BSSA*, 91(2), pp.347-364. 
- Chambers, D., Jin, G., Tourei, A., Issah, A.H.S., Lellouch, A., Martin, E.R., Zhu, D., Girard, A.J., Yuan, S., Cullison, T. and Snyder, T., 2024. Dascore: A python library for distributed fiber optic sensing. *Seismica*, 3(2), pp.10-26443.


In [ ]:
# Packages installation if not already installed

# !pip install dascore --upgrade --quiet
# !pip install PyQt5 --upgrade --quiet

In [3]:
import os
import numpy as np
from scipy.interpolate import CubicSpline

import dascore as dc
from dascore.units import Hz

from matplotlib import pyplot as plt

# Use Qt backend for interactive plots
%matplotlib qt

### Reading and Preprocessing DAS Data

*Note: You can skip this section if you're using the provided example dataset (`example.h5`).*

In [ ]:
# Define file path and filename for DAS data
folder_path = 'Path/To/Your/DAS/Folder'  # Replace with your actual folder path
filename = 'data.hdf5'  # Replace with your actual DAS file name

full_path = os.path.join(folder_path, filename)

# Read and preprocess data
pa = dc.spool(full_path)
pa_fil = (
    pa[0]
    .set_units("1/s", distance="m", time="s")  # Set physical units
    .detrend("time")                           # Remove linear trend
    .decimate(time=20)                         # Downsample in time
    .taper(time=0.05)                          # Apply taper in time
    .pass_filter(time=(1 * Hz, 49 * Hz))       # Bandpass filter 
    .select(distance=(102, 222), samples=True) # Select a distance range
    .transpose('distance', 'time')             
)

### Using the Example Preprocessed Data

The file `example.h5` contains data that has already been preprocessed using the steps described above.


In [5]:
# Read the preprocessed example data and visualize it
pa_fil = dc.spool('./example.h5')

# Plot the preprocessed data
fig, ax = plt.subplots(figsize=(7, 5))
pa_fil[0].viz.waterfall(ax=ax,scale=0.0001)

<Axes: xlabel='time', ylabel='distance(m)'>

### Velocity Spectral (p-f) Analysis

This section performs slowness-frequency transformation on the DAS data.  
The workflow includes:
- Chunking the data into overlapping time windows.
- Applying tapers in both time and distance to reduce edge effects.
- Performing a tau-p (slant stack) transform to map the data into the slowness domain.
- Computing the discrete Fourier transform (DFT) along the time axis to obtain the frequency content for each slowness.
- The resulting spectra are used for subsequent stacking, normalization, and dispersion curve picking.

In [6]:
# Define processing functions for each step
def taper(patch, time, window_type):
    """Apply a taper along the time axis."""
    return patch.taper(time=0.1, window_type=window_type)

def taper_d(patch, distance, window_type):
    """Apply a taper along the distance axis."""
    return patch.taper(distance=0.1, window_type=window_type)

def taup(patch, velocities):
    """Apply tau-p (slant stack) transform with given velocities."""
    return patch.tau_p(velocities)

def dft(patch, dim="time"):
    """Apply discrete Fourier transform along the specified dimension."""
    return patch.dft(dim="time", real=True)


# Process the data step by step
sp_fil = dc.spool(pa_fil)

# Chunk the data into segments of 30 seconds with 10 seconds overlap
sp_fil_chunked = sp_fil.chunk(time=30, overlap=10)

# Apply tapering in time
sp_fil_chunked_taper = dc.spool(
    sp_fil_chunked.map(taper, time=0.1, window_type="hann")
)

# Apply tapering in distance
sp_fil_chunked_taper_d = dc.spool(
    sp_fil_chunked_taper.map(taper_d, distance=0.1, window_type="hann")
)

# Apply tau-p transform with specified velocities
sp_fil_chunked_taper_taup = dc.spool(
    sp_fil_chunked_taper_d.map(taup, velocities=np.arange(100, 800, 20))
)

# Apply discrete Fourier transform along the time dimension
sp_fil_chunked_taper_taup_dft = dc.spool(
    sp_fil_chunked_taper_taup.map(dft, dim="time")
)

Output()

Output()

Output()

Output()

Output()

Output()

Output()

Output()

### Spectral normalization and stacking

This section computes the normalized and stacked power spectrum across all data chunks.  
For each chunk, the code:
- Calculates the power spectrum (magnitude squared of the DFT).
- Splits the spectrum into positive and negative slowness components and sums them to enforce symmetry.
- Normalizes each frequency by the average amplitude across slowness.
- Stacks (sums) the normalized spectra from all chunks to enhance coherent features and suppress noise.

The result, `summed_avg_stack`, is a 2D array representing the normalized, stacked power as a function of frequency and slowness, ready for visualization and dispersion analysis.

In [ ]:
# Calculate number of frequency bins and midpoint
n_freq = sp_fil_chunked_taper_taup_dft[0].data.shape[0]
half = n_freq // 2

# Initialize the stack for the averaged, normalized spectra
if n_freq % 2 == 0:
    summed_avg_stack = np.zeros((half, sp_fil_chunked_taper_taup_dft[0].data.shape[1]))
else:
    summed_avg_stack = np.zeros((half + 1, sp_fil_chunked_taper_taup_dft[0].data.shape[1]))

# Loop through each chunked patch and sum normalized power spectra
for i, patch in enumerate(sp_fil_chunked_taper_taup_dft):
    
    # Compute power spectrum 
    power_spectrum = (patch.data * np.conj(patch.data)).real

    if n_freq % 2 == 0:
        # Split into negative and positive slowness components
        # Even: includes Nyquist frequency
        neg = power_spectrum[:half,:]
        pos = power_spectrum[half:,:]
        neg_flipped  = np.flipud(neg)
        summed = pos + neg_flipped
    else:
        # Split into negative and positive slowness components
        # Odd
        pos = power_spectrum[:half+1]
        neg = power_spectrum[half+1:]
        neg_flipped  = np.flipud(neg)
        summed = pos + neg_flipped[:pos.shape[0]]

    # Normalize by the average across slowness for each frequency
    avg = np.sum(summed, axis=0) / len(summed)
    summed_avg = summed/avg

    # Accumulate the normalized spectra
    summed_avg_stack = summed_avg_stack + summed_avg

### Rayleigh Phase-Velocity Dispersion Picking

This section enables interactive picking of the Rayleigh wave dispersion curve from the stacked slowness-frequency (p-f) spectrum.  

- The normalized and stacked power spectrum is displayed as an image, with frequency on the x-axis and apparent velocity on the y-axis.
- The user manually selects points along the visible dispersion curve by clicking on the plot; pressing Enter finishes the selection.
- The picked points are sorted and interpolated using a cubic spline to produce a smooth dispersion curve.
- The result is visualized by overlaying the picked points and the interpolated curve on the power spectrum image.

This process allows for extraction of the fundamental mode Rayleigh wave phase velocity dispersion curve for subsequent 1D shear-wave velocity inversion.

In [ ]:
# Get frequency and slowness arrays
freq = sp_fil_chunked_taper_taup_dft[0].get_coord('ft_time')[:summed.shape[1]]
slowness = sp_fil_chunked_taper_taup_dft[0].get_coord('slowness')[half:]

# Create figure
fig, ax = plt.subplots(figsize=(10, 5))
extent = [freq[0], freq[-1], 1/slowness[0], 1/slowness[-1]] # Frequency and apparent velocity extent
im = ax.imshow(
    summed_avg_stack**2,
    aspect='auto',
    origin='lower',
    extent=extent,
    cmap='plasma',
    vmin=0,
    vmax=np.max(summed_avg_stack**2)*0.02  # Adjust for better visibility
)
ax.set_xlabel('Frequency (Hz)', fontsize=14)
ax.set_ylabel('Apparent Velocity (m/s)', fontsize=14)
ax.set_title('Click to pick points, then press Enter', fontsize=14)
ax.set_xlim(1, 20)
ax.set_ylim(100, 600)
fig.colorbar(im, ax=ax, label='Amplitude')

# Pick points on that same figure
print("Click to pick points along the dispersion curve. Press Enter when done.")
picked = fig.ginput(n=-1, timeout=0)  # block until Enter

plt.close(fig)

picked = np.array(picked)
if picked.shape[0] < 2:
    print("Not enough points picked for interpolation.")
else:
    # Sort and interpolate
    picked = picked[np.argsort(picked[:, 0])]
    cs = CubicSpline(picked[:, 0], picked[:, 1])
    freq_interp = np.linspace(picked[:, 0].min(), picked[:, 0].max(), 200)
    velocity_interp = cs(freq_interp)

    # Plot result with cubic spline interpolation
    fig2, ax2 = plt.subplots(figsize=(10, 5))
    im2 = ax2.imshow(
        summed_avg_stack**2,
        aspect='auto',
        origin='lower',
        extent=extent,
        cmap='plasma',
        vmin=0,
        vmax=np.max(summed_avg_stack**2)*0.02 
    )
    ax2.set_xlabel('Frequency (Hz)', fontsize=14)
    ax2.set_ylabel('Apparent Velocity (m/s)', fontsize=14)
    ax2.set_title('Manual Picked Dispersion and Cubic Spline Interpolation', fontsize=14)
    ax2.set_xlim(1, 20)
    ax2.set_ylim(100, 600)
    fig2.colorbar(im2, ax=ax2, label='Amplitude')
    ax2.plot(picked[:, 0], picked[:, 1], 'ro', label='Picked Points')
    ax2.plot(freq_interp, velocity_interp, 'b-', linewidth=2, label='Cubic Spline')
    ax2.legend()
    plt.show()


In [ ]:
# Plot static image of the summed power spectrum
# Get frequency and slowness arrays
freq = sp_fil_chunked_taper_taup_dft[0].get_coord('ft_time')[:summed.shape[1]]
slowness = sp_fil_chunked_taper_taup_dft[0].get_coord('slowness')[half:]

plt.figure(figsize=(10, 5))
extent = [freq[0], freq[-1],  1/slowness[0], 1/slowness[-1],]
plt.imshow(
    summed_avg_stack**2,
    aspect='auto',
    origin='lower',
    extent=extent,
    cmap='plasma',
    vmin=0,
    vmax=np.max(summed_avg_stack**2)*0.02
)
plt.plot(picked[:, 0], picked[:, 1], 'ro', label='Picked Points')
plt.plot(freq_interp, velocity_interp, 'b-', linewidth=2, label='Cubic Spline')
plt.xlabel('Frequency (Hz)',fontsize=14)
plt.ylabel('Apparent Velocity (m/s)',fontsize=14)
plt.title('Summed Power Spectrum',fontsize=14)
plt.colorbar(label='Amplitude')
plt.xlim(1,20)
plt.ylim(100,600)
plt.show()


# Correlation-Enhanced ReMi (Ce‑ReMi) for DAS

This section adds an *efficient* correlation-enhanced alternative to the classic delay‑and‑sum ReMi implemented above.  
Key features:

- **Phase‑only/whitened normalization** per window to suppress non‑stationary bursts.  
- **Correlation‑domain beamforming** on a linear DAS array using either:
  - **Coherence‑weighted delay‑and‑sum** (fast, default), or  
  - **MVDR/Capon** with light diagonal loading (more selective; optional via `method="mvdr"`).  
- **Signed slowness** to separate along‑line directions (±p).

The implementation is API‑compatible with the notebook’s use of DASCore `Patch` objects and runs on the **chunked** windows created earlier (`sp_fil_chunked`).  
You can use it with standard geophone spreads or DAS (after decimating to a sensible effective spacing).


In [1]:

import numpy as np
from typing import Tuple, Dict, Optional

def _get_coord_array(patch, name: str):
    '''
    Best-effort helper to get a coordinate array from a DASCore Patch.
    Tries several common access patterns for robustness.
    '''
    # DASCore Patch has .coords with mapping-like access
    if hasattr(patch, "coords"):
        try:
            arr = patch.coords[name]
            try:
                # Some coords entries may be objects with .values/.data
                return np.asarray(getattr(arr, "values", getattr(arr, "data", arr)))
            except Exception:
                return np.asarray(arr)
        except Exception:
            pass
        # getattr-style
        try:
            arr = getattr(patch.coords, name)
            return np.asarray(getattr(arr, "values", getattr(arr, "data", arr)))
        except Exception:
            pass
    # Fallbacks
    if hasattr(patch, name):
        arr = getattr(patch, name)
        return np.asarray(getattr(arr, "values", getattr(arr, "data", arr)))
    raise AttributeError(f"Could not find coordinate '{name}' on patch. Available: "
                         f"{list(getattr(patch, 'coords', {}))}")

def _get_data_2d(patch):
    '''
    Return (data, x, t) where data is 2D as [distance, time].
    Ensures the orientation used in this notebook.
    '''
    # Ensure correct orientation per earlier cell: transpose('distance','time') was applied
    data = patch.data if hasattr(patch, "data") else np.asarray(patch)
    data = np.asarray(data)
    x = _get_coord_array(patch, "distance")
    t = _get_coord_array(patch, "time")
    # If shapes mismatch, try to transpose
    if data.shape[0] != x.size or data.shape[1] != t.size:
        # Try transpose
        if data.shape[1] == x.size and data.shape[0] == t.size:
            data = data.T
        else:
            raise ValueError(f"Unexpected data shape {data.shape} for coords (Nx={x.size}, Nt={t.size}). "
                             "Make sure to call patch.transpose('distance','time') beforehand.")
    return data, x, t

def _phase_only_fft(window_data: np.ndarray, axis: int = -1, eps: float = 1e-12) -> np.ndarray:
    '''
    Compute FFT along time axis and apply phase-only normalization per channel.
    Returns complex spectrum with unit magnitude (except eps guarding).
    '''
    spec = np.fft.rfft(window_data, axis=axis)
    mag = np.abs(spec)
    spec_po = spec / (mag + eps)
    return spec_po

def _freq_axis_rfft(dt: float, n: int) -> np.ndarray:
    return np.fft.rfftfreq(n, d=dt)

def _steering(distance: np.ndarray, freq: np.ndarray, p: np.ndarray) -> np.ndarray:
    '''
    Build steering vectors a(f,p) with shape [F, P, Nch].
    a_{f,p,n} = exp(-i 2π f p x_n).
    '''
    # shapes: F x 1 x 1, 1 x P x 1, 1 x 1 x N
    F = freq[:, None, None]
    P = p[None, :, None]
    X = distance[None, None, :]
    return np.exp(-1j * 2.0 * np.pi * F * P * X)

def ce_remi_beamform_on_window(patch_window,
                               p_grid: np.ndarray,
                               fmin: Optional[float]=None,
                               fmax: Optional[float]=None,
                               method: str="coh",   # "coh" or "mvdr"
                               diag_load: float=1e-2) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    '''
    Compute a p-f power image for one window Patch using correlation-enhanced ReMi.

    Parameters
    ----------
    patch_window : DASCore Patch oriented as [distance, time]
    p_grid       : slowness grid in s/m (with sign for direction)
    fmin, fmax   : frequency band (Hz)
    method       : 'coh' (coherence-weighted delay-sum via phase-only) or 'mvdr'
    diag_load    : diagonal loading factor for MVDR (as fraction of trace(R)/N)

    Returns
    -------
    power_pf : array [F, P]
    freq     : array [F] Hz
    p_grid   : array [P] s/m
    '''
    # Extract arrays
    X, x, t = None, None, None
    X, x, t = _get_data_2d(patch_window)
    dt = float(np.median(np.diff(t)))
    # Phase-only spectrum per channel
    U = _phase_only_fft(X, axis=1)  # shape [Nch, F]
    freq = _freq_axis_rfft(dt, X.shape[1])  # [F]

    # Band-limit
    fmask = np.ones_like(freq, dtype=bool)
    if fmin is not None:
        fmask &= freq >= fmin
    if fmax is not None:
        fmask &= freq <= fmax
    freq = freq[fmask]
    U = U[:, fmask]  # [Nch, F]
    Nch, F = U.shape[0], freq.size

    # Steering vectors
    A = _steering(x, freq, p_grid)  # [F, P, Nch]

    if method.lower() == "coh":
        # Coherence-weighted / phase-only delay-and-sum:
        # y(f,p) = a^H(f,p) * U_phase_only(f)  => power = |y|^2
        # Arrange shapes: U^* conjugate is applied by a^H; compute via einsum
        # U is [Nch, F] => swap to [F, Nch]
        Uf = np.swapaxes(U, 0, 1)  # [F, Nch]
        y = np.einsum("fpn,fn->fp", np.conjugate(A), Uf, optimize=True)
        power_pf = np.abs(y)**2 / (Nch**2)
    elif method.lower() == "mvdr":
        # Build covariance R(f) = < U U^H > (phase-only spectrum used)
        # For a single window, this is rank-1; MVDR needs loading.
        # For better stability, you can pass in multiple windows and average externally.
        power_list = []
        for iF in range(F):
            u = U[:, iF][:, None]  # [Nch, 1]
            R = (u @ u.conj().T)  # [Nch, Nch]
            # Diagonal loading using trace heuristic
            alpha = diag_load * (np.trace(R).real / max(Nch, 1))
            R_loaded = R + alpha * np.eye(Nch, dtype=R.dtype)
            # Solve R^{-1} a for all p at once
            # A_f is [P, Nch] (transpose of A[iF])
            A_f = A[iF]  # [P, Nch]
            try:
                # Cholesky for Hermitian positive-definite
                L = np.linalg.cholesky(R_loaded)
                # Solve R^{-1}A via two triangular solves
                z = np.linalg.solve(L, A_f.T)        # solve L z = A_f^T
                RinvA = np.linalg.solve(L.conj().T, z)  # solve L^H x = z
            except np.linalg.LinAlgError:
                # Fall back to pseudo-inverse (should be rare with loading)
                RinvA = np.linalg.pinv(R_loaded) @ A_f.T
            denom = np.einsum("pn,pn->p", A_f.conj(), RinvA.T).real  # a^H R^{-1} a for all p
            # Avoid divide by zero
            denom = np.maximum(denom, 1e-12)
            Pf = 1.0 / denom  # [P]
            power_list.append(Pf)
        power_pf = np.vstack(power_list)  # [F, P]
        # Normalize for display
        power_pf /= (power_pf.max() + 1e-12)
    else:
        raise ValueError("method must be 'coh' or 'mvdr'")

    return power_pf, freq, p_grid

def ce_remi_beamform(spool_chunked,
                     p_grid: np.ndarray,
                     fmin: Optional[float]=None,
                     fmax: Optional[float]=None,
                     method: str="coh",
                     diag_load: float=1e-2,
                     average: str="median") -> Dict[str, np.ndarray]:
    '''
    Run Ce-ReMi over all chunked windows and aggregate a single p-f image.

    Parameters
    ----------
    spool_chunked : a DASCore spool of windows (e.g., sp_fil_chunked)
    p_grid        : slowness grid (s/m), including sign for direction separation
    fmin, fmax    : band in Hz
    method        : 'coh' (default) or 'mvdr'
    diag_load     : diagonal loading factor for MVDR
    average       : 'median' or 'mean' aggregation across windows

    Returns
    -------
    dict with keys: {'power_pf', 'freq', 'p', 'power_pf_windows'}
    - power_pf: aggregated [F, P]
    - freq: frequency axis used [F]
    - p: slowness axis [P]
    - power_pf_windows: list/array of per-window images for uncertainty/QA
    '''
    images = []
    freq_common = None
    for idx, patchw in enumerate(spool_chunked):
        # Ensure orientation matches earlier steps
        if hasattr(patchw, "transpose"):
            patchw = patchw.transpose("distance", "time")
        power_pf, freq, p_out = ce_remi_beamform_on_window(
            patchw, p_grid, fmin=fmin, fmax=fmax, method=method, diag_load=diag_load
        )
        if freq_common is None:
            freq_common = freq
        else:
            # Align by interpolation if needed
            if not np.array_equal(freq_common, freq):
                from numpy import interp
                new_img = np.empty((freq_common.size, p_out.size), dtype=power_pf.dtype)
                for j in range(p_out.size):
                    new_img[:, j] = interp(freq_common, freq, power_pf[:, j], left=0.0, right=0.0)
                power_pf = new_img
        images.append(power_pf)

    if not images:
        raise RuntimeError("No windows found in spool_chunked. Did you compute sp_fil_chunked?")

    stack = np.stack(images, axis=0)  # [W, F, P]
    if average == "median":
        agg = np.median(stack, axis=0)
    else:
        agg = np.mean(stack, axis=0)
    return {
        "power_pf": agg,
        "freq": freq_common,
        "p": p_out,
        "power_pf_windows": stack,
    }


In [13]:
freq[0], freq[-1]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [14]:

# === Ce‑ReMi demo on the chunked spool ===
# Requirements: previously created `sp_fil_chunked` in this notebook.

# Define slowness grid with sign (s/m). Example: 0.001 to 0.01 s/m (100–1000 m/s) both directions
p_abs = np.linspace(1/1000.0, 1/100.0, 120)  # 1000 to 100 m/s
p_grid = np.concatenate((-p_abs[::-1], p_abs))

# Frequency band (match your earlier bandpass settings)
fmin, fmax = 1.0, 40.0

# Run fast coherence-weighted Ce‑ReMi
try:
    ce_out = ce_remi_beamform(sp_fil_chunked, p_grid, fmin=fmin, fmax=fmax, method="coh", average="median")
    power_pf = ce_out["power_pf"]
    freq = ce_out["freq"]
    p = ce_out["p"]
except Exception as e:
    print("Ce‑ReMi demo could not run:", e)
    power_pf = None

# Plot if available
if power_pf is not None:
    velocity = 1.0 / (np.maximum(np.abs(p), 1e-6))
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(10,5))
    extent = [fmin, fmax, velocity.min(), velocity.max()]
    im = ax.imshow(power_pf.T, origin="lower", aspect="auto", extent=extent, cmap="plasma")
    ax.set_xlabel("Frequency (Hz)")
    ax.set_ylabel("Apparent Velocity (m/s)")
    ax.set_title("Ce‑ReMi (coherence-weighted)")
    fig.colorbar(im, ax=ax, label="Power (a.u.)")
    ax.set_ylim(100, 1000)  # adjust to your site
    plt.show()


/var/folders/0f/qh8vgw8n4g570f0r6q17pdz00000gn/T/ipykernel_40075/7689541.py:12: UserWarning: Currently coords[coord_name] returns a numpy array, but in a future dascore version it will return a BaseCoord instance. Use patch.coords.get_array(coord_name) instead.
  arr = patch.coords[name]
/var/folders/0f/qh8vgw8n4g570f0r6q17pdz00000gn/T/ipykernel_40075/7689541.py:12: UserWarning: Currently coords[coord_name] returns a numpy array, but in a future dascore version it will return a BaseCoord instance. Use patch.coords.get_array(coord_name) instead.
  arr = patch.coords[name]
/var/folders/0f/qh8vgw8n4g570f0r6q17pdz00000gn/T/ipykernel_40075/7689541.py:12: UserWarning: Currently coords[coord_name] returns a numpy array, but in a future dascore version it will return a BaseCoord instance. Use patch.coords.get_array(coord_name) instead.
  arr = patch.coords[name]
/var/folders/0f/qh8vgw8n4g570f0r6q17pdz00000gn/T/ipykernel_40075/7689541.py:12: UserWarning: Currently coords[coord_name] returns a n

In [16]:
ce_out

{'power_pf': array([], shape=(0, 240), dtype=float64),
 'freq': array([], dtype=float64),
 'p': array([-0.01      , -0.00992437, -0.00984874, -0.00977311, -0.00969748,
        -0.00962185, -0.00954622, -0.00947059, -0.00939496, -0.00931933,
        -0.0092437 , -0.00916807, -0.00909244, -0.00901681, -0.00894118,
        -0.00886555, -0.00878992, -0.00871429, -0.00863866, -0.00856303,
        -0.00848739, -0.00841176, -0.00833613, -0.0082605 , -0.00818487,
        -0.00810924, -0.00803361, -0.00795798, -0.00788235, -0.00780672,
        -0.00773109, -0.00765546, -0.00757983, -0.0075042 , -0.00742857,
        -0.00735294, -0.00727731, -0.00720168, -0.00712605, -0.00705042,
        -0.00697479, -0.00689916, -0.00682353, -0.0067479 , -0.00667227,
        -0.00659664, -0.00652101, -0.00644538, -0.00636975, -0.00629412,
        -0.00621849, -0.00614286, -0.00606723, -0.0059916 , -0.00591597,
        -0.00584034, -0.00576471, -0.00568908, -0.00561345, -0.00553782,
        -0.00546218, -0.00538